# Лабораторная работа №2

Цель работы - изучить аналитический (обратной матрицы) и численный (Брауна-Робинсона) методы нахождения смешанных стратегий в антагонистической игре двух лиц в нормальной форме.

## Постановка задачи

Найдите цену игры и оптимальные стратегии обоих игроков методами обратной матрицы и Брауна-Робинсон. Сравните полученные результаты.

## Ход работы

Вариант 5.

In [1]:
pretty_print_default(True)
latex.add_to_preamble("\\usepackage[russian]{babel}")
from IPython.display import display, Math, Latex

Зададим матрицу игры:

In [2]:
C = matrix(SR, 3, 3, [8, 12, 10, 1, 6, 19, 17, 11, 11])
C

[ 8 12 10]
[ 1  6 19]
[17 11 11]

Стратегии игрока $A$:

In [3]:
x = C.rows()
x

[(8, 12, 10), (1, 6, 19), (17, 11, 11)]

Стратегии игрока $B$:

In [4]:
y = C.columns()
y

[(8, 1, 17), (12, 6, 11), (10, 19, 11)]

### Аналитический метод

In [5]:
u = matrix(SR, 1, C.nrows(), [1] * C.nrows())
v = (1/(u * C^(-1) * u.transpose()))
v

[845/76]

Оптимальные стратегии игрока $A$:

In [6]:
x_star = v * (u * C^(-1))
x_star

[39/76  3/38 31/76]

Оптимальные стратегии игрока $B$:

In [7]:
y_star = v * (C^(-1) * u.transpose()).transpose()
y_star

[ 3/152  11/19 61/152]

### Метод Брауна-Робинсон

In [8]:
k = 12 # Число итераций

xk, yk = 0, 0

winA = [0] * C.nrows()
lossB = [0] * C.ncols()

vxk, vyk = dict(), dict()
for i in range(C.nrows()):
    vxk[i], vyk[i] = 0, 0

rows = [['$k$', 'Выбор игрока $A$', 'Выбор игрока $B$', 'Выигрыш $A$', 'Проигрыш $B$', '$\overline{v}^{k}$', '$\underline{v}^{k}$', '$\\varepsilon$']]
for ki in range(1, k + 1):
    winA = [xi + yi for xi, yi in zip(winA, y[yk])]
    lossB = [xi + yi for xi, yi in zip(lossB, x[xk])]

    maxX = max(winA)
    minY = min(lossB)

    vxk[xk] += 1
    vyk[yk] += 1

    rows.append([ki, xk + 1, yk + 1, winA, lossB, maxX / ki, minY / ki, maxX / ki - minY / ki])

    countX = winA.count(maxX)
    countY = lossB.count(minY)

    xk = winA.index(maxX)
    yk = lossB.index(minY)

    if countX > 1 and countY > 1:
        raise Exception("Should not be at all")
    elif countX == 1 and countY > 1:
        if lossB[xk] == minY:
            yk = xk
    elif countY == 1 and countX > 1:
        if winA[yk] == maxX:
            xk = yk
table(rows)

,Выбор игрока,Выбор игрока,Выигрыш,Проигрыш,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,


Тогда оптимальные стратегии для игрока $A$:

In [9]:
x_strategies = matrix(1, 3, [i / k for i in vxk.values()])
x_strategies

[1/12 5/12  1/2]

Оптимальные стратегии для игрока $B$:

In [10]:
y_strategies = matrix([i / k for i in vyk.values()])
y_strategies

[1/6 1/3 1/2]